# FC 온라인 인벤 크롤링

## 데이터 수집 환경 조성

### 필요 라이브러리 호출

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
# from openpyxl import Workbook
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.action_chains import ActionChains

# from konlpy.tag import Kkma, Komoran, Okt, Hannanum # 토큰화
# from konlpy.tag import Mecab # 토큰화
# from collections import Counter
# from wordcloud import WordCloud
# import nltk
# import re

import json
from time import sleep
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] =False

# Warnings 제거
import warnings
warnings.filterwarnings('ignore')

---

## 데이터 크롤링

### 1) 분석 실시할 선수 카드 입력

In [2]:
# 선수 이름, 시즌 카드 받아오기
player_search = input("선수 이름을 입력해주세요.")
season_icon_number = input("시즌 아이콘을 선택해주세요.")
season_icon = input("시즌 아이콘을 선택해주세요.")

In [3]:
# 선수 이름, 시즌 카드 확인
print(player_search)
print(season_icon_number)
print(season_icon)

손흥민
280
22 UEFA Champions League


### 2) 데이터 스크래핑 진행(BeautifulSoup)

- 선수 리뷰 페이지 url에서 page, searchword, season 변수에 값을 변경하며 조회

- 먼저 첫 페이지 html을 크롤링한 후, 마지막 페이지 번호를 받아온다.

- 비어있는 comment_list를 생성하고 반복문을 통해 마지막 페이지까지 유저의 선수 리뷰를 스크래핑한 후에 comment_list에 추가한다.

In [4]:
# 기본 첫페이지 선택
page = 1

In [5]:
# FC온라인 인벤 선수 평가 페이지 가져오기
url = f'https://fconline.inven.co.kr/dataninfo/rate/index.php?pg={page}&searchword={player_search}&season={season_icon_number}'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=EDGE" http-equiv="X-UA-Compatible"/>
  <meta content="FC 온라인,FCO,Fifa online4,선수 DB,비교,평가,공략,세레모니,페이스온,넥슨,선수 평가" name="keywords"/>
  <meta content="FC 온라인 정보, FCO, 피파 온라인4, Fifa4, 뉴스, 스쿼드 시뮬레이터, 선수 정보, 팁과 노하우, 전술 상담, 개인기, 영상관, 커뮤니티 제공" name="description"/>
  <meta content="3a508505e9bf09f6b8b7abe782e34f3cc895eb65" name="naver-site-verification"/>
  <meta content="fifaonline4" name="inven-site"/>
  <meta content="FC 온라인" name="inven-sitename"/>
  <meta content="FC 온라인 인벤 - 선수 평가" property="og:title"/>
  <meta content="website" property="og:type"/>
  <meta content="FC 온라인 정보, FCO, 피파 온라인4, Fifa4, 뉴스, 스쿼드 시뮬레이터, 선수 정보, 팁과 노하우, 전술 상담, 개인기, 영상관, 커뮤니티 제공" property="og:description"/>
  <meta content="DkdFwQGro9RDeuncf0XmKA==" id="inven-verification" name="inven-verification"/>
  <title>
   FC 온라인 인벤 - 선수 평가
  </title>
  <link href="https://www.inven.co.kr/common/

In [6]:
# 마지막 페이지 넘버 받아오기
pages = soup.select('span.basetext > a.pg')
last_page = pages[-1].get_text()
last_page

'10'

In [7]:
# 선수 리뷰 리스트 생성
comment_list = []

# 마지막 페이지까지 크롤링
for page in range(1, int(last_page)+1):
    url = f'https://fconline.inven.co.kr/dataninfo/rate/index.php?pg={page}&searchword={player_search}&season={season_icon_number}'
    
    # 페이지 comment 받아오기
    comments = soup.select('span.fifa4.comment')
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    
    # 페이지별 comment 리뷰 리스트에 추가하기
    for comment in comments:
        comment_list.append(comment.get_text())


In [8]:
# 총 리뷰 개수
len(comment_list)

200

In [9]:
# 총 리뷰 리스트
comment_list

['금카 사용 중이고, 콘차붐이랑 투톱으로 기용하거나 공미로 사용 중인데중거리면 중거리감차면 감차그냥 개사기다한가지 단점이 있다면 슛이 너무 좋으니까 자꾸 그것만 하려는 습관이 생겨버림...',
 '금카 아직 현역이다 7조대 가성비 금흥민감차 치달 퍼터 중거리 미쳤음 체감까지 괜찮고 금카케미까지 받을 수 있어서 메리트 있음진짜 솔직히 24토티은카 8조짜리랑 차이가 없음걍 진짜 좋음..',
 '금카 사용중CAP보다는 확실히 우위의 22시즌 나머지 120오버롤 대비해서 다 거기서 거기이고 이게 제일 가겨대비 성능도 좋음딱 윙어에 10조 이상 금카 흥민이 쓰는건 조금 낭비임이걸로 솔직히 흥민이는 FC온라인 종결임',
 '\t24토티+5 쓰다가 급여 때문에 이거 금카로 넘어왔는데 솔직히 블라인드 테스트하면 못 맞출 것 같음 비슷비슷한 듯',
 '다른시즌에 비해서는 모르겠는데 22챔 금카 공미로두고 중거리 개맛있게먹는중 궤적이 진짜다름 금카 개좋음진짜윗시즌금카부터는 14조라서 이게 가성비 국밥임',
 '3년째 챔스거주중 키보드유저 캡금이랑 다를거없고 몸싸움이랑 드리블은 여전히 너무 안좋다 바로 이적시장침투',
 '금카 한번 써보고 싶습니다 부탁드립니다 형님들!',
 '금카 하한 전재산 제발팔아주세요ㅕ',
 '24토티기 훨씬 좋음',
 '금카 상한 알박 3일째 제발 팔아주세요,.,,',
 '이거 금카 캐미받고 톱 어떤가요',
 '현역케미가 없는데유',
 '요 며칠간 24토티 손흥민 은카 사용해보니까 23토티, 22챔스 8카보다 위치선정, 침투가 좋았음참여도 3/3이 아니라 3/2인 느낌임인벤 데이터 베이스를 어떻게 분석하는 지는 모르겠지만본가 손흥민은 참여도 3/2임22 챔스 금카 손흥민 , 24토티 손흥민 둘다 사용해본 결과 24토티 손흥민 5카로 정착함이유는 24토티가 확실히 똑똑했고 골 포인트가 더 높았고1대1 상황에서 24토티가 더 안정적임어시스트 횟수도 24토티 손흥민 5카가 확실히 많았음24토티는 st , cf , lw , rw로 사용했을때 패마로 특성단 22